In [13]:
import dask.dataframe as dd
import duckdb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests

from scripts.pyensembl_operations import import_pyensembl
g37 = import_pyensembl(37)


real_files = "read_parquet('results/processed/real/0/*.parquet')"
synth_files = "read_parquet('results/processed/synth/**/*.parquet')"

real = dd.read_parquet("results/processed/real/0/*.parquet")
synth = dd.read_parquet("results/processed/synth/**/*.parquet")

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


# getting upreg/downreg amounts

In [8]:
query_real = f"""
    SELECT 
        gene_id,
        COUNT(*) FILTER (WHERE is_gene_upregulated = TRUE) as upregulated,
        COUNT(*) FILTER (WHERE is_gene_upregulated = FALSE) as downregulated
    FROM {real_files}
    GROUP BY gene_id
"""

result_real = duckdb.sql(query_real).df()

query_synth = f"""
    SELECT 
        gene_id,
        COUNT(*) FILTER (WHERE is_gene_upregulated = TRUE) as upregulated,
        COUNT(*) FILTER (WHERE is_gene_upregulated = FALSE) as downregulated
    FROM {synth_files}
    GROUP BY gene_id
"""


result_synth = duckdb.sql(query_synth).df()



# taking the crossection and merging the dfs together

In [9]:
common_genes = list(set(result_real.gene_id)  & set(result_synth.gene_id))
result_real = result_real[result_real.gene_id.isin(common_genes)]
result_synth = result_synth[result_synth.gene_id.isin(common_genes)]

# divide synth amounts by 10
result_synth["upregulated"] = result_synth["upregulated"] / 10
result_synth["downregulated"] = result_synth["downregulated"] / 10

# remove not_found
result_real = result_real[result_real.gene_id != "not_found"]
result_synth = result_synth[result_synth.gene_id != "not_found"]

values = pd.merge(result_real, result_synth, on="gene_id", suffixes=["_real", "_synth"])


# log2odds

In [10]:
def calculate_log2_odds_ratio(a, b, c, d, k=0.5):
    # a, b, c, d are the four cells of the 2x2 contingency table
    # k is the smoothing constant
    #
    odds_ratio = ((a + k) * (d + k)) / ((b + k) * (c + k))
    return np.log2(odds_ratio)

# laplace smoothing with k=0.5 (Jeffreys prior)
values['log2_odds_ratio'] = values.apply(lambda row: calculate_log2_odds_ratio(
    row['upregulated_real'], 
    row['downregulated_real'], 
    row['upregulated_synth'], 
    row['downregulated_synth']
), axis=1)

values

,gene_id,upregulated_real,downregulated_real,upregulated_synth,downregulated_synth,log2_odds_ratio
0,ENSG00000198586,2741,1457,3337.6,1559.0,-0.186474
1,ENSG00000227367,1392,736,985.7,484.8,-0.104083
2,ENSG00000233080,2126,590,1833.1,636.8,0.323838
3,ENSG00000153827,1958,1016,2305.2,1217.4,0.025330
4,ENSG00000225889,1590,881,2086.9,852.9,-0.438965
...,...,...,...,...,...,...
31423,ENSG00000223330,2,58,0.8,23.2,-0.360133
31424,ENSG00000124196,0,15,9.5,6.1,-5.553658
31425,ENSG00000232925,0,35,5.8,16.3,-4.734710
31426,ENSG00000187747,19,0,41.6,2.2,1.322613


# fisher's exact

In [12]:
def perform_fisher_test_vectorized(df, pseudocount=0.5):
    # Add pseudocount to the table
    table = np.array([
        [df['upregulated_real'] + pseudocount, df['downregulated_real'] + pseudocount],
        [df['upregulated_synth'] + pseudocount, df['downregulated_synth'] + pseudocount]
    ]).transpose((2, 0, 1))  # reshape for 2x2 tables

    p_values = np.zeros(len(df))

    for i in range(len(df)):
        _, p_values[i] = fisher_exact(table[i])

    df['p_value'] = p_values
    df['p_adj'] = multipletests(p_values, method='fdr_bh')[1]
    
    return df
def add_z_score(df):
    # Calculate mean and standard deviation of log2 odds ratios
    mean_log2or = df['log2_odds_ratio'].mean()
    std_log2or = df['log2_odds_ratio'].std()
    
    # Calculate Z-score
    df['z_score'] = (df['log2_odds_ratio'] - mean_log2or) / std_log2or
    
    return df


df = perform_fisher_test_vectorized(values)
df = add_z_score(df)
df.head()

,gene_id,upregulated_real,downregulated_real,upregulated_synth,downregulated_synth,log2_odds_ratio,p_value,p_adj,z_score
0,ENSG00000198586,2741,1457,3337.6,1559.0,-0.186474,3.952448e-03,0.014360,-0.264947
1,ENSG00000227367,1392,736,985.7,484.8,-0.104083,3.163041e-01,0.494740,-0.201436
2,ENSG00000233080,2126,590,1833.1,636.8,0.323838,5.930840e-04,0.002862,0.128432
3,ENSG00000153827,1958,1016,2305.2,1217.4,0.025330,7.530856e-01,0.909248,-0.101676
4,ENSG00000225889,1590,881,2086.9,852.9,-0.438965,1.925104e-07,0.000003,-0.459583


# adding pyensembl stuff

In [14]:
# Create dictionaries for both gene names and biotypes
gene_names = {gene.gene_id: gene.gene_name for gene in g37.genes()}
biotypes = {gene.gene_id: gene.biotype for gene in g37.genes()}

# Add both columns to the DataFrame
df["gene_name"] = df["gene_id"].map(gene_names)
df["biotype"] = df["gene_id"].map(biotypes)

df.head()


,gene_id,upregulated_real,downregulated_real,upregulated_synth,downregulated_synth,log2_odds_ratio,p_value,p_adj,z_score,gene_name,biotype
0,ENSG00000198586,2741,1457,3337.6,1559.0,-0.186474,3.952448e-03,0.014360,-0.264947,TLK1,protein_coding
1,ENSG00000227367,1392,736,985.7,484.8,-0.104083,3.163041e-01,0.494740,-0.201436,SLC9B1P4,pseudogene
2,ENSG00000233080,2126,590,1833.1,636.8,0.323838,5.930840e-04,0.002862,0.128432,CTA-714B7.5,lincRNA
3,ENSG00000153827,1958,1016,2305.2,1217.4,0.025330,7.530856e-01,0.909248,-0.101676,TRIP12,protein_coding
4,ENSG00000225889,1590,881,2086.9,852.9,-0.438965,1.925104e-07,0.000003,-0.459583,AC074289.1,antisense


In [15]:
df.to_csv("results/the_final_data.csv", index=False)
